In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

We are now going to take on the Titanic dataset, and predict wheter a given passanger will survive or not - given a few features.

Make sure to read about the dataset [here.](https://www.kaggle.com/c/titanic/data)

In [ ]:
# load the titanic dataset

titanic_df = pd.read_csv('../data/titanic.csv')

titanic_df

In [ ]:
import seaborn as sns

In [ ]:
titanic_df.Sex.value_counts()

In [ ]:
titanic_df[titanic_df.Sex == 'male']['Survived'].value_counts()

In [ ]:
titanic_df[titanic_df.Sex == 'female']['Survived'].value_counts()

In [ ]:
titanic_df.Sex.isnull().sum()

---

## Challenges

**Task 1**

Read the documentation of the dataset, and think carefully about the features available. Which ones do you think are relevant to predict wether someone survives or not?

Now drop the columns you think are redundant.

In [ ]:
titanic_df = titanic_df.drop(columns=['PassengerId', 'Ticket', 'Cabin', 'Name'])

In [ ]:
titanic_df

In [ ]:
titanic_df.info()

**Task 2**

Do some preliminary EDA, and clean columns in a suitable way if warranted. Remember to also conduct (if needed) feature engineering, one-hot-encoding (categorical columns) etc.

In general, prepare your data for training.

*Note*: You do NOT need to do any feature scaling for logistic regression.

In [ ]:
titanic_df = pd.get_dummies(titanic_df, columns=['Embarked'], dtype=int) 

titanic_df['Sex'] = [1 if sex == 'male' else 0 for sex in titanic_df['Sex']]

In [ ]:
titanic_df

Alis exempel ovan. Obs, man kan jobba MYCKet mer på detta men nu är åtminstone alla features numeriska

Obs, age har None's i sig! Det måste ni lösa

In [ ]:
# vi fyller alla våra NaNs i age med medelvärdet av övriga värden. Obs, detta är INTE bästa sättet att fylla i på

mean_age = titanic_df['Age'].mean()

print('mean age:', mean_age)

titanic_df['Age'].fillna(mean_age, inplace=True)    # inplace = True gör att vi ändrar på titanic_df, utan att behöva re-assigna

Vi skalar också våra vår Age och Fare

titanic_df['Age'] = titanic_df['Age'] / max(titanic_df['Age'])

titanic_df['Fare'] = titanic_df['Fare'] / max(titanic_df['Fare'])

**Task 3**

Train a logistic regression model to predict wether someone survives, or not. 

Use 5-fold cross-validation to assess performance.

Use accuracy, precision and recall score.

In [ ]:
X, y = titanic_df.drop(columns=['Survived']), titanic_df['Survived']

titanic_df = titanic_df.drop(columns=['Age'])

X = X.drop(columns=['Age'])

In [ ]:
X

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score

# Initialize linear regression model
model = LogisticRegression()

# Define the scoring function
scoring = {'accuracy': make_scorer(accuracy_score), 
           'precision': make_scorer(precision_score), 
           'recall': make_scorer(recall_score)}

# Perform 5-fold cross-validation
num_folds = 5

cv_results = cross_validate(model, X, y, cv=num_folds, scoring=scoring, return_train_score=True)

In [ ]:
cv_results

In [ ]:
np.mean(cv_results['test_accuracy'])

In [ ]:
np.mean(cv_results['test_precision'])

In [ ]:
np.mean(cv_results['test_recall'])

**Task 4**

Now do a normal train/test split and train a logistic model on this.

Plot a confusion matrix on the predictions of your test set. Is your model performing okay-ish?

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X.values,
                                                    y.values,
                                                    test_size=test_size, 
                                                    random_state=42)

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

In [ ]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()

#train the model
logistic_model.fit(X_train, y_train)

In [ ]:
# vi kan använda predict för att direkt få ut den mest sannolika klassen

y_train_hats = logistic_model.predict(X_train)
y_test_hats = logistic_model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(y_true=y_test, y_pred=y_test_hats)

cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0, 1])

cm_display.plot()
plt.show() 